In [ ]:
import os

import numpy as np
import tensorflow.compat.v2 as tf

import matplotlib
import matplotlib.pyplot as plt

# necesario para mostrar las imagenes
%matplotlib inline  

from google.protobuf import text_format

from object_detection import model_lib_v2
from object_detection import exporter_lib_v2
from object_detection.protos import pipeline_pb2
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

from PIL import Image, ImageDraw, ImageFont

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)

tf.enable_v2_behavior()
tf.get_logger().setLevel('ERROR')           # Suppress TensorFlow logging (2)


# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

CONFIG_FILE = 'models/efficientdet_d0/pipeline.config'
DIR_MODEL_TRAINING = 'models/efficientdet_d0'
DIR_MODEL_EXPORTED = 'exported-models/my_model'
PATH_TO_SAVED_MODEL = DIR_MODEL_EXPORTED + "/saved_model"
LABEL_MAP_FILE = 'annotations/label_map.pbtxt'

# Ejecución de entrenamiento

El comando en linea es

python model_main_tf2.py --model_dir=models/efficientdet_d0 --pipeline_config_path=models/efficientdet_d0/pipeline.config

In [ ]:
strategy = tf.compat.v2.distribute.MirroredStrategy()
with strategy.scope():
    model_lib_v2.train_loop(
        pipeline_config_path=CONFIG_FILE,
        model_dir=DIR_MODEL_TRAINING)


## Exportar el modelo

El comando en linea es 

python ./exporter_main_v2.py --input_type image_tensor --pipeline_config_path ./models/my_ssd_resnet50_v1_fpn/pipeline.config --trained_checkpoint_dir ./models/my_ssd_resnet50_v1_fpn/ --output_directory ./exported-models/my_model

In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_FILE, 'r') as f:
    text_format.Merge(f.read(), pipeline_config)
text_format.Merge('', pipeline_config)
exporter_lib_v2.export_inference_graph(
    'image_tensor', pipeline_config, DIR_MODEL_TRAINING,
    DIR_MODEL_EXPORTED)

# Probar el modelo

In [ ]:
# Load saved model and build the detection function
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(LABEL_MAP_FILE, use_display_name=True)

In [ ]:
image_np = np.array(Image.open("../data/19-104.jpg"))
input_tensor = tf.convert_to_tensor(image_np)
input_tensor = input_tensor[tf.newaxis, ...]
detections = detect_fn(input_tensor)

In [ ]:
image_np = np.array(Image.open("../data/19-104.jpg"))
input_tensor = tf.convert_to_tensor(image_np)
input_tensor = input_tensor[tf.newaxis, ...]
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
                for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

image_np_with_detections = image_np.copy()

In [ ]:
%matplotlib inline 


viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections,
      detections['detection_boxes'],
      detections['detection_classes'],
      detections['detection_scores'],
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=.030,
      agnostic_mode=False)

plt.figure(figsize= (20,10))
plt.imshow(image_np_with_detections)
plt.show()

In [61]:
int(
    category_index.get(
        detections['detection_classes'][0]
        ).get('name')
    )



8001304263